# HW 4 Answers

NAME: Eric Wu

In [1]:
#mydir = '/home/jkinser/Documents/courses/CDSCardinal/Version5/'
#mydir = '/Users/jkinser/Documents/CoursesKinser/CDSCardinal/Version5/'

mydir = '/Users/eric/Documents/CDS465/'

pysrc = mydir + 'pysrc/'
datadir = mydir + 'data/'
popdir = mydir + 'population/'

import sys
sys.path.append( pysrc )

import pandas as pd
import numpy as np

In [2]:
import Main
E = Main.Realm( datadir )
#E.ReadAll( popdir + 'mpop07.db')
popname = 'pop2_5.db'
E.ReadAll( popdir + popname )
len(E.persondf)

56062

# Task 1 (3 pts)
How many immigrants also have a jail record? Merge _portdf_ and _jaildf_ on a citizen's _pid_ value. Print the number of immigrants and the number of those with jail records.

In [3]:
# Your code
#identify imigrants who also hage jail records

#from portdf keep only pid col and imigrants

s1 = E.portdf['emigdate'] < 0
criminal_imigrants = pd.merge(E.portdf.loc[s1]['pid'], E.jaildf['pid'],on='pid')

#outputs
print(f'The number of imigrants is {len(E.portdf)}')
print(f'The number of imigrants who have jail records is {len(criminal_imigrants)}')

The number of imigrants is 1080
The number of imigrants who have jail records is 27


# Task 2 (3 pts)
Find the five oldest citizens who are living and were never married.

In [4]:
# Your code
#where clauses
#needs oldest citizens
#needs to be alive
#needs to not be married

#id married persons
married_pids = pd.concat([E.weddf['husband'],E.weddf['wife']],ignore_index=True).unique()

s1 = E.persondf[~E.persondf['pid'].isin(married_pids)]['pid'] #id rows that are not in the array
s2 = E.persondf['death'] < 0 
query = s1 & s2

"""
sanity check
print(E.persondf.query('pid==26'))
print('\n')
print(E.weddf[E.weddf['husband']==26])
"""

cols = ['pid', 'birth', 'death', 'gender', 'spouse', 'firstname', 'lastname']
E.persondf.loc[query][cols].groupby(['birth']).min().head(5)

/var/folders/9v/q1p9lnr906n75d765h87_j9h0000gn/T/ipykernel_72739/2762444717.py:12: FutureWarning: Operation between non boolean Series with different indexes will no longer return a boolean result in a future version. Cast both Series to object type to maintain the prior behavior.
  query = s1 & s2


,pid,death,gender,spouse,firstname,lastname
birth,,,,,,
6440,50573,-1,M,-1,Calloway,Jeon
6445,50855,-1,M,-1,Karan,Kang
6460,50673,-1,F,-1,Hina,Gwok
6469,51133,-1,M,-1,Jacee,Pan
6489,51511,-1,M,-1,Axcel,Ha


# Task 3 (4 pts)
What percentage of immigrant marriages are inter-racial? (Make life easier and consider only one gender - husbands or wives). Consider all marriages.

Print:
* The number of marriages involving male (or female) immigrants
* The number of marriages which are inter-racial
* The ratio of these two - percentage of immigrant marriages which are inter-racial

1. Print: The number of marriages involving male (or female) immigrants

In [5]:
#isolate imigrants
imigrants = E.portdf['pid']

#identify pids of married imigrants
#merge over husband to find wives & drop husband since we merged over it
#merge over wife to find husbands & drop wife since we merged over it
married_imigrants = pd.merge(
                            left=pd.merge(
                                left=imigrants,
                                right=E.weddf[['husband', 'wife']],
                                left_on='pid',
                                right_on='husband',
                                how='left'
                            ).drop('husband', axis=1), 
                        right=E.weddf[['husband', 'wife']],
                        left_on='pid',
                        right_on='wife',
                        how='left',
                        suffixes=('','_h')
                    ).drop('wife_h',axis=1)

#outputs
print(f'The number of male imigrants that got married is {married_imigrants['wife'].count()}')
print(f'The number of female imigrants that got married is {married_imigrants['husband'].count()}')

The number of male imigrants that got married is 120
The number of female imigrants that got married is 126


2. Print: The number of marriages which are inter-racial

In [6]:
#query imigrants that have gotten married
query = (married_imigrants['husband'] > 0) | (married_imigrants['wife'] > 0)
married_imigrants = married_imigrants.loc[query].copy()

#create 'spouse' column
married_imigrants['spouse'] = np.where(married_imigrants['husband'] > 0, married_imigrants['husband'], married_imigrants['wife'])

#sanity check
#married_imigrants['spouse'].count()

#identify interacial marriages 
ntr_marriages = pd.merge(
                    left=pd.merge(
                        left=married_imigrants[['pid','spouse']],
                        right=E.persondf[['pid','dometh']], 
                        on='pid'
                    ), 
                    right=E.persondf[['pid','dometh']], 
                    left_on='spouse', 
                    right_on='pid', 
                    suffixes=('','_s')
                ).drop('pid_s',axis=1)

#print inter racial marriages
query = (ntr_marriages['dometh'] != ntr_marriages['dometh_s'])
print(f'The number of interacial marriages is {ntr_marriages.loc[query]['pid'].count()}')

The number of interacial marriages is 30


3. Print: The ratio of these two - percentage of immigrant marriages which are inter-racial

In [ ]:
#male imigrants
print(ntr_marriages.loc[query]['pid'].count()/married_imigrants['wife'].count())

0.25


In [ ]:
#female imigrants
print(ntr_marriages.loc[query]['pid'].count()/married_imigrants['husband'].count())

0.23809523809523808


In [ ]:
#both genders
imigrant_marriage_cnt = married_imigrants['pid'].count()
ntr_imigrant_marriage_cnt = ntr_marriages.loc[query]['pid'].count()

print(f'The ratio of interacial to non interacial marriages in imigrants is {ntr_imigrant_marriage_cnt/imigrant_marriage_cnt}')

The ratio of interacial to non interacial marriages in imigrants is 0.12195121951219512
